### Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación

### Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones 2021
Búsqueda y Recomendación para Textos Legales

Mentor: Jorge E. Pérez Villella

# Práctico Introducción al Aprendizaje Automático

Integrantes:

- Christian Oviedo
- Francisco Correa

El objetivo de este práctico es probar distintos modelos de clasificación para evaluar la performance y la exactitud de predicción de cada modelo. 

* Utilizando el corpus normalizado en el práctico anterior, transformar el texto en vectores numéricos utilizando scikit-learn comparando los 3 modelos de vectorización. Explicar cada uno estos modelos.

* Clasificar los documentos por fuero. Trabajaremos con los siguientes modelos de clasificación de la librería scikit-learn: Logistic Regresion, Naive Bayes y SVM. En cada modelo probar distintos hiperparámetros, generar la Matriz de Confusión y la Curva ROC. Explicar los resultados obtenidos.

* Determinar y justificar cual es el modelo con mejor performance y predecir el fuero de un documento utilizando el mejor modelo.


Fecha de Entrega: 15 de agosto de 2021

Se carga el corpus normalizado en en el práctico anterior

In [12]:
#!conda install -y yellowbrick

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it



from sklearn import linear_model
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import HashingVectorizer

from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from sklearn.metrics import roc_auc_score

from sklearn import svm, datasets

from sklearn.ensemble import RandomForestClassifier

Esta clase permite hacer un gridsearch sobre diferentes modelos. Ver http://www.davidsbatista.net/blog/2018/02/23/model_optimization/ 

In [14]:
corpus_file_name = 'cleaned_corpus.csv'
cleaned_corpus = pd.read_csv(corpus_file_name)

In [15]:
cleaned_corpus.head()

,Unnamed: 0,text,id,classifier
0,0,dato causa sede ciudad cordoba dependencia juz...,4de122c24ab1606c9d67f4ff9e656143,Documentos/MENORES
1,1,univoco fecha materia revista familia tribunal...,1f9cdcb2c2596656b540c1271fc2d843,Documentos/MENORES
2,2,juzgado juventud violencia familiar 8ª cordoba...,17dcae14592fc6e87680ccb4251d9395,Documentos/MENORES
3,3,auto caratulado a. a. denuncia violencia gener...,4b3ae58648b6267ebb332feec8002588,Documentos/MENORES
4,4,juzg adolescencia violencia familiar 4ta cba s...,1316026beaa1d7e6530bdfe7e54f7b5c,Documentos/MENORES


Generamos los sets de entrenamiento y de testing

In [16]:
X = cleaned_corpus['text']  
y = cleaned_corpus['classifier']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Modelos de Vectorización

Esta función recibe una lista de documentos para entrenar (X_train), una lista de documentos para hacer el testing (X_test )del modelo entrenado, y un vectorizer para transformar a vectores los documentos de entrenamiento y prueba. Devuelve dos matrices sparse donde cada fila es un vector que representa un documento.

In [17]:
def get_vectors(X_train, X_test, vectorizer):
    
    X_train_vect = vectorizer.fit_transform(X_train)
    X_text_vect = vectorizer.transform(X_test)
    
    return (X_train_vect, X_text_vect)

## Vectorización con CountVectorizer (tambien conocido como One-hot encoding)
Este esquema de vectorización es el más simple y básico. Se genera un vector que representa todas las palabras del corpus. Cada documento es representado como una instancia del vector anterior indicando la cantidad de veces que aparece cada palabra. Notar que este modelo de vectorización esta sesgado para el caso en que tengamos palabras poco frecuentes pero muy significativas para clasificar documentos y también para palabras que aparezcan 'mucho', pero que aparezca en todos los documentos. En el último caso, si tratamos de diferenciar los documentos en base a las palabras que los componen, que las palabras aparezcan en todos ellos, no aporta información.

Como se explica en el parrafo anterior, es el modelo mas sencillo, generando por cada un token un vector de dimensionalidad exacta a la longitud del vocabulario con un 1 en su dimension y 0 en todos los otros lugares.

![One-hot](./images/one-hot.png)

Para trabajar con matrices mas pequeñas scikit-learn en su modelo CountVectorizer() nos retorna matrices esparsas (es posible tambien obtener la matriz dispersa u "original" correspondiente)

In [18]:
count_vect = CountVectorizer()

X_train_vect , X_test_vect = get_vectors(X_train, X_test, count_vect)

X_train_vect.shape

(162, 15696)

In [19]:
print ("X_train")
print (f"Cantidad de documentos:{X_train_vect.shape[0]}, Cantidad de dimensiones por documento: {X_train_vect.shape[1]} ")

print ("X_test")
print (f"Cantidad de documentos:{X_test_vect.shape[0]}, Cantidad de dimensiones por documento: {X_test_vect.shape[1]} ")

X_train
Cantidad de documentos:162, Cantidad de dimensiones por documento: 15696 
X_test
Cantidad de documentos:81, Cantidad de dimensiones por documento: 15696 


In [20]:
#count_vect.vocabulary_

## Vectorización con HashingVectorizer

El método anterior, aunque es sencillo, su implemetacion guarda el vocabulario "in-memory" en forma de diccionario, lo cual puede ser causante de un gran consumo de memoria si el dataset es extenso.

A resolver este problema viene el siguiente método: HashingVectorizer. Este método convierte una serie de documentos a una matriz de ocurrencias de tokens (como CountVectorizer) pero aplicando a cada feature una funcion de hash y usando este valor como indice en lugar de usar los indices "normales" de forma asociativa. Usa el algoritmo conocido como "hashing trick". More info: https://en.wikipedia.org/wiki/Feature_hashing.

Este método presenta las ventajas de permitir escabilidad en terminos de uso de memoria, facil uso de pickle para los vectores y/0 matrices resultantes y la posibilidad de usarlo en "fit" parciales (mas conocido como streaming, de aqui la posibilidad de trabajar con datasets enormes). 

Aunque existen algunas desventajas como que pueden existir "colisiones de hashes" (features distintas mapeadas a indices iguales) siendo que el numero de features es un parametro y el equipo de desarrollo deberia elegir un numero fijo, esto no es en si algo malo, siendo que dependiendo del caso escoger un n_features mas pequeño reduciria la complejidad de nuestro modelo. Y tambien existe el problema de que la conversion de un token a hash es unidireccional lo cual puede ser un problema si luego del entramiento del modelo se quisiera inspeccionar las features que serian mas importantes.

In [21]:
n_features = 3000
hash_vect = HashingVectorizer(n_features=n_features)
X_train_hash , X_test_hash = get_vectors(X_train, X_test, hash_vect)

print(X_train_hash.shape, X_test_hash.shape)
print(X_train_hash.toarray())

(162, 3000) (81, 3000)
[[ 0.          0.          0.         ...  0.          0.03912554
   0.        ]
 [ 0.          0.         -0.0058243  ...  0.          0.0058243
   0.        ]
 [ 0.          0.          0.         ...  0.          0.0493147
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.01537688
   0.        ]
 [ 0.          0.          0.00494257 ...  0.          0.06919594
   0.        ]
 [ 0.          0.          0.00982424 ...  0.          0.03929698
   0.        ]]


## Vectorización con TfidfVectorizer
Cuando vimos el método CountVectorizer, vimos que no es suficiente únicamente contar la cantidad de veces que aparece una palabra en un documento, puesto que no es lo mismo que esa palabra aparezca en el resto de los documentos (en este caso, la palabra no brinda información relevante) a que aparezca en un número reducido de documentos (en este caso la palabra si brinda información relevante.)
Es por eso que es necesario contar con algún método que permita hacer este tipo de distinciones a la hora de expresar documentos como vectores.

El método TF-IDF (Term Frequency – Inverse Documento Frequency) busca generar vectores que indican no solamente cuanto aparece una palabra en un documento, sino que tan frecuente es esa palabra en el resto de los documentos.

TF(palabra,documento) =  $\frac{\textrm{Número de veces que la palabra aparece en el documento}}{\textrm{Número de palabras diferentes en el documento}}$


DF(palabra,documentos) =  $\frac{\textrm{Cantidad de documentos que tienen la palabra}}{\textrm{cantidad de documentos}}$


IDF(palabra,documentos) = $ log ({\frac{\textrm{cantidad de documentos}}{\textrm{Cantidad de documentos que tienen la palabra}}})$

Notar que en IDF, cuando la cantidad de documentos que tienen la palabra, se acerca a la cantidad de documentos, el resultado de la división se acerca a 1 y por consiguiente, el logartimo a 0.

**TF - IDF = (palabra, documentos, documento) = TF (palabra, documento) $\times$ IDF (palabra, documentos)**

Scikit-learn ofrece 2 maneras de realizar esto: TfidfVectorizer y TfidfTransformer, el primero se aplica sobre la serie de documentos. Dependiendo del problema, si se necesitara obtener el conteo de frecuencia de tokens para realizar otras tareas entonces ir por la opcion CountVectorizer+TfidfTransformer es mejor idea.

In [22]:
# Using TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf , X_test_tfidf = get_vectors(X_train, X_test, vectorizer)

print(X_train_tfidf.shape) 

# Using TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True,use_idf=True)

X_train_tfidf_transformer = transformer.fit(X_train_vect)

print(X_train_tfidf_transformer.idf_.shape) # Solo obteniendo los valores idf.

(162, 15696)
(15696,)


In [23]:
print ("X_train")
print (f"Cantidad de documentos:{X_train_tfidf.shape[0]}, Cantidad de dimensiones por documento: {X_train_tfidf.shape[1]} ")

print ("X_test")
print (f"Cantidad de documentos:{X_test_tfidf.shape[0]}, Cantidad de dimensiones por documento: {X_test_tfidf.shape[1]} ")

X_train
Cantidad de documentos:162, Cantidad de dimensiones por documento: 15696 
X_test
Cantidad de documentos:81, Cantidad de dimensiones por documento: 15696 


In [24]:
#vectorizer.vocabulary_

Las variables X_train_counts y X_test_counts se usan para todos los cálculos que siguen. Estas variables las igualamos a las variables que tienen los resultados de TF – IDF por ser el mejor método de vectorización de los tres analizados. En el caso de querer correr la notebook con el resultado de CountVectorizer, o Hash, descomentar las celdas correspondientes

In [25]:
X_train_counts  = X_train_tfidf
X_test_counts = X_test_tfidf

In [26]:
#X_train_counts  = X_train_vect
#X_test_counts = X_test_vect

# Clasificación usando diferentes modelos

## Curva ROC


Se puede utilizar un gráfico AUC ROC (Área bajo la curva de características operativas del receptor) para visualizar el rendimiento de un modelo entre la sensibilidad y la especificidad. La sensibilidad se refiere a la capacidad de identificar correctamente las entradas que pertenecen a la clase positiva. La especificidad se refiere a la capacidad de identificar correctamente las entradas que pertenecen a la clase negativa. Dicho de otra manera, una gráfica AUC ROC puede ayudar a identificar qué tan bien su modelo es capaz de distinguir entre clases.


En los problemas del mundo real, a menudo hay una superposición entre las clases, lo que significa que detectar todos los verdaderos negativos y verdaderos positivos puede ser desafío muchas veces de imposible solución. Se muestra a continuación una ilustración de lo que se esta comentando:


<img src="images/ROC_curve.png">



Notar que dependiendo donde se ubique el umbral de predicción, cambiará el número de TP, TN, FP y FN. Notar también que en el gráfico anterior, las distribuciones se solapan (lo que sucede con frecuencia en problemas de la vida real), lo que trae como consecuencia, tener inevitablemente FN y FP.

El tipo de problemática que quiera resolver, es el que indicará hacia donde muevo el umbral. Supongamos por ejemplo que lo que estamos tratando de clasificar es si un paciente tiene cáncer o no. Es claro que en este ejemplo, no quiero tener falsos negativos (FNwww.www). No es aceptable que a una persona que tiene cáncer, le de un diagnóstico en el cual indica que esta sana. En este caso, voy a mover el umbral de predicción hacia la izquierda de manera tal que no exista la posibilidad de generar falsos negativos (FN). Al mover el umbral de esta manera, voy a aumentar inevitablemente la cantidad de falsos positivos (FP), los cuales luego puedo descartar con estudios más específicos.

Volviendo gráfico AUC ROC, una puntuación AUC de 1 significa que el modelo puede distinguir con precisión entre las dos clases el 100% del tiempo, es decir, estamos frente a un clasificador ideal. Una puntuación de 0,5 significa que el modelo no puede determinar entre las dos clases y, en esencia, está adivinando. La curva ROC es la gráfica de la tasa de verdaderos positivos del modelo frente a la tasa de falsos positivos.



$\textrm{TPR} =  \frac{\textrm{TP}} {{\textrm{TP}}➕{\textrm{FN}} }$

$\textrm{FPR} =  \frac{\textrm{FP}} {{\textrm{FP}}➕{\textrm{TN}} }$


De lo antes indicado, entonces al momento de evaluar los modelos de clasificación, buscaremos o nos quedaremos con aquel modelo y los correspondientes híper parámetros para los cuales el área bajo la curva ROC sea lo más cercana a 1.

<img src="images/ROC_curve_values.png">

La clasificación que se solicita en la mentoria, es una clasificación multi clase, es por ello que debemos extender el concepto de curvas ROC a curvas ROC para clasificaciones multi clases

Lo que se estuvo presentando hasta el momento esta orientado a clasificaciones binarias. Como se puede hacer para llevar o reutilizar lo antes mencionado en clasificaciones multi clases. Una estrategia para lograr esto, es que las curvas ROC se pueden trazar con la metodología de usar una clase frente al resto. Utilizando uno contra el resto para cada clase, se tendrá el mismo número de curvas que clases. La puntuación AUC también se puede calcular para cada clase individualmente.


<img src="images/ROC_curve_multiclass.png">

El componente que utilizaremos para realizar la Curva ROC para clasificaciones multi clases, es el provisto por  **yellowbrick** (https://www.scikit-yb.org/en/latest/api/classifier/rocauc.html)


ROC Multiclass: https://medium.com/swlh/how-to-create-an-auc-roc-plot-for-a-multiclass-model-9e13838dd3de

La función **test_model** recibe un modelo de clasificación (por ejemplo Naive Bayes, Regresión Logística, etc ), los sets de entrenamiento y testing, y muestra:


-	La métricas: precisión, recall, f1-score
-	La matriz de confusión
-	Curca ROC Multiclase y el AUC correspondiente

Este método usa el método plot_ROC_curve para poder visualizar la curva ROC multiclase


In [27]:
from yellowbrick.classifier import ROCAUC

def plot_ROC_curve(model, xtrain, ytrain, xtest, ytest , ax , macro = True, micro = True , per_class = True):

    # Creating visualization with the readable labels
    visualizer = ROCAUC(estimator = model , ax = ax , macro = macro, micro = micro , per_class = per_class)
                                        
    # Fitting to the training data first then scoring with the test data                                    
    visualizer.fit(xtrain, ytrain)
    visualizer.score(xtest, ytest)
    visualizer.show()
    
    #return visualizer


def test_model(model, X_train, y_train, X_test, y_test ):
    
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)
    print(metrics.classification_report(y_test, y_test_pred ))

    
    fig, ax = plt.subplots(figsize=(10, 6))
    plt.grid(False)
    ax.set_title('Confusion Matrx')

    disp =metrics.plot_confusion_matrix(model, X_test, y_test, display_labels= ["FAMILIA" ,"LABORAL" , "MENORES" , "PENAL" ], ax = ax)
    
    #disp =metrics.plot_confusion_matrix(model, X_test, y_test, display_labels= ["FAMILIA" ,"LABORAL" , "MENORES" , "PENAL" ])
    

    #disp.confusion_matrix
    
    fig1, ax1 = plt.subplots(figsize=(10, 6))
    ax1.set_title('ROC')
        
    plot_ROC_curve(model = model, xtrain = X_train, ytrain = y_train, xtest = X_test, ytest = y_test , ax = ax1 )
    

ModuleNotFoundError: No module named 'yellowbrick'

Seteamos el seed para que todos los experimentos sean repetibles

In [ ]:
seed = 42

Los modelos y los híper paramétros mostrados a continuación, provienen de los resultados obtenidos en el apartado **Anexo**, punto *Optimización de modelos*.  Ahí se pueden observar los modelos que se probaron, los híper parámetros utilizados y los resultados obtenidos. 

## Logistic Regression

In [ ]:
ltest = linear_model.LogisticRegression(multi_class= 'ovr',solver = 'liblinear', random_state= seed)

test_model(ltest,X_train_counts, y_train, X_test_counts, y_test )

In [ ]:
lm = linear_model.LogisticRegression(multi_class='ovr', solver='liblinear' , random_state = seed)

test_model(lm,X_train_counts, y_train, X_test_counts, y_test )

## Naive Bayes

In [ ]:
nb = MultinomialNB()
test_model(nb,X_train_counts, y_train, X_test_counts, y_test )

## SVM

In [ ]:
rbf = svm.SVC(kernel='rbf' , random_state = seed)

test_model(rbf,X_train_counts, y_train, X_test_counts, y_test )


In [ ]:
poly = svm.SVC(kernel='poly', degree=3 , random_state = seed )

In [ ]:
test_model(poly,X_train_counts, y_train, X_test_counts, y_test )

## Resultados

En líneas generales se observa un buen desempeño de todos los modelos, salvo el MultinomialNB. Quizás este buen desempeño se producto de la poca cantidad de documentos y que además los documentos hallan sido generados por el mismo grupo de persona en cada fuero. Es decir, por ejemplo en el fuero penal, quizás los fallos pertenecen al mismo juez y han sido redactados por la misma persona. Esto indicaría un fuerte sesgo en el set de datos. Sugerimos contar con más fallos para poder aumentar el el set de datos y al mismo tiempo darle más variabilidad al mismo.

 Si bien dentro de los modelos de clasificación solicitados, no estaba indicado RandomForest, lo hemos agregado al mismo en el Anexo. De todos los modelos, Random Forest fue el que mejor desempeño tuvo. 

Los mejores modelos y los respectivos híper parámetros encontrados son:

### RandomForest:

-	criterion: entropy
-	n_estimadors: 100
  

### RandomForest:

-	criterion: gini
-	n_estimadors: 100

### SVC:

-	kernel: poly
-	degree: 2

### SVC:

-	kernel: rbf

### SVC:

-	kernel: linear

### SVC:

-	kernel: sigmoid


### LogisticRegression


-	C: 1.0
-	multi_class: ovr
-	penalty: l2
-	solver: liblinear

En el apartado **Anexo**, punto *Optimización de modelos*, se puede observar los modelos que se probaron, los híper parámetros utilizados y los resultados obtenidos. 

In [ ]:
document_id = 30
print (f"Label del documento {y_test.values[document_id]}")

In [ ]:
# El valor por defecto de n_estimatores es 100
rf_entropy = RandomForestClassifier(criterion = 'entropy') 
rf_entropy.fit(X_train_counts, y_train)

In [ ]:
print(f"Predicción {rf_entropy.predict(X_test_counts[document_id])}")

In [ ]:
rf_gini = RandomForestClassifier(criterion = 'gini') 
rf_gini.fit(X_train_counts, y_train)

In [ ]:
print(f"Predicción {rf_gini.predict(X_test_counts[document_id])}")

In [ ]:
svc_poly = svm.SVC(kernel = "poly" , degree = 3 )
svc_poly.fit(X_train_counts, y_train)

In [ ]:
print(f"Predicción {svc_poly.predict(X_test_counts[document_id])}")

In [ ]:
svc_rbf = svm.SVC(kernel = "rbf" )
svc_rbf.fit(X_train_counts, y_train)

In [ ]:
print(f"Predicción {svc_rbf.predict(X_test_counts[document_id])}")

# Anexo

## Optimización de modelos

Realizamos una implementación de gridsearch con cross validation, que permite pasar diferentes modelos de sickit-learn a ajustar. 
La idea es que este método nos permita hacer pruebas de manera sencilla de diferentes métodos con diferentes parámetros. Luego en base a estos resultados, elegimos que modelos y parámetros presentar en el apartado * Clasificación usando diferentes modelos*

A la función **train_modelos** se le pasan:
-	 Dos diccionarios: los modelos y los parámetros.
-	 Los sets de entrenamiento y test
-	 La cantidad de folds

El método hace el entrenamiento de todos los modelos en base a los parámetros que se le indican y usando el CV indicado. La función no calcula por el momento cual es el mejor modelo e hiper parámetros correspondientes. Esto se hace por medio de una inspección visual. En futuras versiones se puede implementar la función última descripta. 


Otra alternativa a esta implementación puede ser la indicada en http://www.davidsbatista.net/blog/2018/02/23/model_optimization/ . Preferimos implementar una versión propia para poder entender más profundamente los conceptos vistos en la diplomatura

In [ ]:
models1 = {
    'RandomForset': RandomForestClassifier(),
    'MultinomialNB': MultinomialNB(),
    'SVM_01': svm.SVC(),
    'SVM_02': svm.SVC(),
    'LogisticRegressionClassifier': linear_model.LogisticRegression() ,
    'LogisticRegressionClassifier_01': linear_model.LogisticRegression()
    
}

params1 = {
    'RandomForset': {"n_estimators" : [100] , "criterion" : ["gini", "entropy"]},
    'LogisticRegressionClassifier': { "solver":["liblinear" , "sag", "saga","lbfgs"], "multi_class":["ovr"], "penalty":["l2" ] , "C": [1.0,0.7]  } ,
    'LogisticRegressionClassifier_01': { "solver":["liblinear" ], "multi_class":["ovr"], "penalty":["l2","l1"] , "C": [1.0,0.7,0.2]  } ,
    'SVM_01':{"kernel" :['poly'] , "degree" : [2,3,4,5] } ,
    'SVM_02':{"kernel" :['linear', 'rbf', 'sigmoid']  } ,
    'MultinomialNB':{"alpha" :[1.0] }
} 


models1T = {
    'SVM_01': svm.SVC(),
  
}

params1T = {
      'SVM_01':{"kernel" :['linear', 'poly', 'rbf', 'sigmoid'] , "degree" : [3] } ,
    
} 

In [31]:
from copy import copy, deepcopy



def roc_auc_score_macro(actual_class, pred_class, average = "macro"):

    roc_auc = roc_auc_score(actual_class, pred_class, average = average , multi_class ='ovr')
 
    return roc_auc



def roc_auc_score_multiclass(actual_class, pred_class, average = "macro"):

  #creating a set of all the unique classes using the actual class list
  unique_class = set(actual_class)
  roc_auc_dict = {}
  for per_class in unique_class:
    #creating a list of all the classes except the current class 
    other_class = [x for x in unique_class if x != per_class]

    #marking the current class as 1 and all other classes as 0
    new_actual_class = [0 if x in other_class else 1 for x in actual_class]
    new_pred_class = [0 if x in other_class else 1 for x in pred_class]

    #using the sklearn metrics method to calculate the roc_auc_score
    roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average , multi_class ='ovr')
    roc_auc_dict[per_class] = roc_auc

  return roc_auc_dict

#def train_model(model, folds_index, X_train, Y_train):

def generate_model_params(model_params):
    
    allNames = sorted(model_params)
    combinations = it.product(*(model_params[Name] for Name in allNames))
    return (list(combinations) , allNames)


def train_model(model, params_names, param_combination, folds_index, X_train, Y_train , X_test, Y_test , output_dict = True , random_state = None):
    
    param_combination = list(param_combination)
    print ("train model")
    print (f"{model} {params_names} {param_combination}")
   
    
    cloned_model = deepcopy(model)
    
    
    for param_name , param_value in zip(params_names,param_combination ):
        #print (f"{param_name} =  {param_value}")
        setattr(cloned_model , param_name , param_value)

    if type(random_state) == int:
        setattr(cloned_model , "random_state" , random_state)
        
    print (cloned_model)
    
    results = []
    
    for train_index, test_index in folds_index:
   
        cloned_model_tmp = deepcopy(cloned_model)
        #print (f"{train_index}")
        #print (f"{test_index}")
    
    
    #X_train, X_test, y_train, y_test
    
        # Se hace el split en base a los CV. Se obtienen los datos de X_train y de X_test con sus respectivos Y
        X_train_tmp, X_test_tmp = X_train[train_index], X_train[test_index]
        
        y_train_tmp, y_test_tmp = Y_train[train_index], Y_train[test_index] 
    
        cloned_model_tmp.fit(X_train_tmp,y_train_tmp)
       
    
        y_test_val_pred = cloned_model_tmp.predict(X_test_tmp)
        
        train_result = metrics.classification_report(y_test_tmp, y_test_val_pred , output_dict = output_dict )
        
        print(train_result)
        
        
        #roc_result = roc_auc_score(y_true = y_test_tmp, y_score = y_test_val_pred , multi_class = "ovr")
        
        roc_result = roc_auc_score_multiclass(actual_class=y_test_tmp, pred_class=y_test_val_pred)
        #roc_result_macro = roc_auc_score_macro(actual_class=y_test_tmp, pred_class=y_test_val_pred)
        
        
        results.append ((f"{model}","Train" , f"{params_names }", train_result , roc_result , f"{param_combination}" ))
    
    
    cloned_model_tmp = deepcopy(cloned_model)
    
    
    cloned_model_tmp.fit(X_train,Y_train)
        
    y_test_pred = cloned_model_tmp.predict(X_test)
    
    test_result = metrics.classification_report(Y_test, y_test_pred , output_dict = output_dict )
    
    #roc_result = roc_auc_score(y_true = Y_test, y_score = y_test_pred , multi_class = "ovr")
    roc_result = roc_auc_score_multiclass(actual_class=Y_test, pred_class=y_test_pred)
    #roc_result_macro = roc_auc_score_macro(actual_class=y_test_tmp, pred_class=y_test_val_pred)
        
    results.append ((f"{model}","Test", f"{params_names} ", test_result , roc_result , f"{param_combination}" ))
    
    print("Test")
    print(test_result)
    
    return results
    
    

def train_models(X_train,Y_train,X_test, Y_test, cv=5,shuffle=True, models=None ,params=None , output_dict = True , random_state = None):
    
    results = []
    
    kf = KFold(n_splits=cv, random_state=random_state, shuffle=shuffle )
   
    
    folds_index = [(train_index, test_index) for train_index, test_index in kf.split(X_train)  ]

    for param_model in params.keys():
    
        params_combination, params_names = generate_model_params(params.get(param_model))
        #print (f"Modelo a ejecutar: {param_model}, parámetros a probar: {params_combination} , nombre de los parámetros: {params_names} ")
        
        for param_combination in params_combination:
            #print (f"{param_model}: {param_combination} ")
            
            model_result = train_model(model = models.get(param_model),params_names = params_names, param_combination = param_combination, folds_index = folds_index, X_train = X_train, Y_train = Y_train , X_test = X_test, Y_test = Y_test , output_dict = output_dict , random_state = random_state )     
            
            results.extend(model_result)
    return results        
       

In [ ]:
#y_train.values

Esta función arma un data frame con el resultado de los entrenamientos. Notar que para calcular el ROC_AUC, se hace una suma de los valores ponderados del ROC por clase

In [ ]:
def toDataFrame(results, y_test):
    
    counter = Counter(y_test)
    total = counter['Documentos/FAMILIA'] + counter['Documentos/LABORAL'] + counter['Documentos/MENORES'] + counter['Documentos/PENAL']
    familia = counter['Documentos/FAMILIA'] / total
    laboral = counter['Documentos/LABORAL'] / total
    menores = counter['Documentos/MENORES'] / total
    penal = counter['Documentos/PENAL'] / total
    
    print ("Ponderado fuero")
    print (f"familia: {familia}, laboral: {laboral}, menores: {menores}, penal: {penal} ")
    
    filtered_values =  []
    columns = ["modelo", "modo" , "parametros" , "valores" , "accuracy", "precision" , "recall" , "f1-score" , "roc_penal", "roc_familia" ,"roc_laboral" , "roc_menores"]
    for result in results:
        #print (f"{result[0]} {result[1]} {result[2]} {result[3]['macro avg']} \n")
        filtered_values.append((result[0], result[1] , result[2] , result[5] , result[3]['accuracy'], result[3]['macro avg']['precision'] , result[3]['macro avg']['recall'] ,  result[3]['macro avg']['f1-score'] , result[4]["Documentos/PENAL"] , result[4]["Documentos/FAMILIA"] , result[4]["Documentos/LABORAL"] , result[4]["Documentos/MENORES"]))

    df= pd.DataFrame(data = filtered_values , columns = columns)
    
    df["roc_ponderado"] = (df["roc_penal"] * penal + df["roc_familia"] * familia + df["roc_laboral"] * laboral + df["roc_menores"] * menores)
    return df

In [ ]:
results  = train_models(X_train= X_train_counts, Y_train =y_train.values ,  X_test = X_test_counts, Y_test = y_test.values,  models = models1 , params = params1 , cv=5 , output_dict = True , random_state = seed )

In [ ]:
result_logistic = toDataFrame(results , y_test.values)

### Ordenamos los modelos (y los parámetros utilizados) según diferentes métricas

#### accuracy y f1-score

In [ ]:
result_logistic[result_logistic["modo"] =="Test"].sort_values(by=['accuracy','f1-score'] , ascending = False)

In [ ]:
#result_logistic.sort_values(by=['roc_promedio','precision'] , ascending = False)

## Random Forest

Si bien el modelo de clasificación random forest no fue solicitado, lo agregamos debido a que los métodos desarrollados nos permiten de una forma sencilla entrenar modelos y visualizar los resultados

In [30]:
seed = 42

randomForest = RandomForestClassifier(criterion="entropy" , random_state = seed)
test_model(randomForest,X_train_counts, y_train, X_test_counts, y_test )

NameError: name 'test_model' is not defined

In [28]:
randomForest = RandomForestClassifier(criterion="gini" , random_state = seed)
test_model(randomForest,X_train_counts, y_train, X_test_counts, y_test )

NameError: name 'seed' is not defined

## Otros modelos de vectorizacion: word2vec

En el mundo de los word embeddings el actual standard de-facto es este modelo de vectorizacion conocido como "word2vec". Scikit-learn no ofrece este metodo pero creimos importante al menos nombrarlo en este practico siendo que es una herramienta de amplio uso en el mundo de NLP actualmente.
Fue creado por Tomas Mikolov en 2013 (en ese momento empleado de Google). Word2vec es un conjunto de redes neuronales entrenados para reconstruir el contexto linguistico de las palabras.

Cabe resaltar que el input de este metodo es igual a los anteriores: el corpus generado desde una serie de documentos.

En metodos como CountVectorizer, se crean espacios n-dimensionales donde n es el tamaño del vocabulario, por lo cual cada palabra tomaria lugar en cada dimesion, haciendo a cada palabra independiente. Eso quiere decir que "bueno" y "genial" estan a la misma distancia (o significan lo mismo) que "perro" por ej, lo cual no es correcto. El objetivo es "agrupar" palabras con contexto similar en posiciones cercanas (tendiendo la similitud de coseno a 1, o sea, el angulo entre estos vectores es cercano a 0).

![One-hot](./images/cos-sim.png)

Word2vec resuelve esto usando una de los 2 siguientes arquitecturas de redes neuronales:
- Skipgram (generalizacion de n-grams): En este caso el input del modelo es la palabra a predecir y el modelo (el modelo interno de w2v) se encarga de retornar el contexto de la misma. 
- CBOW (Continuous bag-of-words): Este caso es al reves del anterior, el input del modelo interno es el contexto de la palabra a predecir.

![One-hot](./images/cbow-sg.png)


Ambos enfoques obviamente presentan sus ventajas y desventajas, pero en resumidas cuentas se podria decir que:
- CBOW tiene una mejor "accuracy" con palabras frecuentes y es bastante mas performante en terminos de tiempo que skip-gram.
- Skip-gram funciona mejor con menores cantidades de datos entrenados (por lo cual es menos performante en tiempo) y ofrece mejor accuracy para palabras "raras" o no tan frecuentes.

En python es posible usar este metodo de word embedding con la ayuda de la libreria 'gensim'. 

In [32]:
# from gensim.models import Word2Vec
